In [ ]:
# ✅ 1. 라이브러리 로드
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType
from datasets import Dataset
import pandas as pd
import torch
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# ✅ 2. 모델 설정 (한국어 특화 모델)
model_name = "beomi/KcBERT-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# ✅ 3. 모델 로딩 (분류용)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
).to("mps")

# ✅ 4. 한국어 이메일 데이터 불러오기 및 전처리
data = pd.read_csv("./spam_dataset_KO_balanced_5000.csv")

def clean_text(text):
    text = re.sub(r"[\\n\\r]", " ", text)
    text = re.sub(r"\\s+", " ", text)
    return text.strip()

data["text"] = data["text"].apply(clean_text)
data["label"] = data["label"].astype(int)

# ✅ 5. 훈련/검증 데이터 분할
train, test = train_test_split(data, test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train[["text", "label"]])
test_dataset = Dataset.from_pandas(test[["text", "label"]])

/Users/tilon/Desktop/Tilon/myvenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/tilon/Desktop/Tilon/myvenv/lib/python3.13/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/KcBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
# ✅ 6. 토큰화
def preprocess(example):
    tokenized = tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=256,
    )
    tokenized["label"] = int(example["label"])
    return tokenized

tokenized_train_dataset = train_dataset.map(preprocess)
tokenized_test_dataset = test_dataset.map(preprocess)

# ✅ 7. LoRA 구성 (KcBERT 구조에 맞는 target_modules)
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS
)
model = get_peft_model(model, peft_config)

# ✅ 8. 평가 지표 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary', zero_division=0)
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }


Map: 100%|██████████| 1000/1000 [00:00<00:00, 9754.94 examples/s]


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/tilon/Desktop/Tilon/myvenv/lib/python3.13/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [3]:
# ✅ 9. 학습 설정
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,  # 🔼 더 오래 학습
    per_device_train_batch_size=8,  # 🔽 과적합 방지
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    warmup_ratio=0.1,  # 🔥 학습 초반엔 천천히 시작
    learning_rate=2e-5,  # 🔽 너무 크지 않게
    weight_decay=0.01,
    logging_steps=10,
    save_steps=100,
    save_total_limit=1,
    optim="adamw_torch",
    report_to="none",
    no_cuda=True
)


# ✅ 10. Trainer 정의 및 학습 실행
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
trainer.can_return_loss = True
trainer.train()

/Users/tilon/Desktop/Tilon/myvenv/lib/python3.13/site-packages/transformers/training_args.py:1376: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
  1%|          | 10/1250 [00:56<1:46:39,  5.16s/it]

{'loss': 0.7479, 'grad_norm': 2.6148362159729004, 'learning_rate': 1.6000000000000001e-06, 'epoch': 0.04}


  2%|▏         | 20/1250 [01:45<1:41:06,  4.93s/it]

{'loss': 0.7369, 'grad_norm': 3.3786673545837402, 'learning_rate': 3.2000000000000003e-06, 'epoch': 0.08}


  2%|▏         | 30/1250 [02:34<1:39:39,  4.90s/it]

...

 99%|█████████▉| 1240/1250 [1:34:21<00:48,  4.80s/it]

{'loss': 0.0026, 'grad_norm': 0.062128663063049316, 'learning_rate': 1.777777777777778e-07, 'epoch': 4.96}


100%|██████████| 1250/1250 [1:35:08<00:00,  4.57s/it]

{'loss': 0.0027, 'grad_norm': 0.059130970388650894, 'learning_rate': 0.0, 'epoch': 5.0}
{'train_runtime': 5708.8426, 'train_samples_per_second': 3.503, 'train_steps_per_second': 0.219, 'train_loss': 0.13890601873099803, 'epoch': 5.0}


TrainOutput(global_step=1250, training_loss=0.13890601873099803, metrics={'train_runtime': 5708.8426, 'train_samples_per_second': 3.503, 'train_steps_per_second': 0.219, 'train_loss': 0.13890601873099803, 'epoch': 5.0})

In [5]:
# ✅ 11. 성능 평가 및 저장
eval_result = trainer.evaluate()
print("\\n📊 평가 지표:")
for key, value in eval_result.items():
    print(f"{key}: {value:.4f}")

trainer.model.save_pretrained("./KO_results")
tokenizer.save_pretrained("./KO_results")

100%|██████████| 250/250 [01:46<00:00,  2.34it/s]


\n📊 평가 지표:
eval_loss: 0.0016
eval_accuracy: 1.0000
eval_f1: 1.0000
eval_precision: 1.0000
eval_recall: 1.0000
eval_runtime: 107.5325
eval_samples_per_second: 9.3000
eval_steps_per_second: 2.3250
epoch: 5.0000


/Users/tilon/Desktop/Tilon/myvenv/lib/python3.13/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('./KO_results/tokenizer_config.json',
 './KO_results/special_tokens_map.json',
 './KO_results/vocab.txt',
 './KO_results/added_tokens.json',
 './KO_results/tokenizer.json')